### Bronze Layer

In [0]:
from pyspark.sql.functions import input_file_name, regexp_extract

df = spark.read \
    .option("header",True) \
    .csv("abfss://parameter-file@databricksdl7.dfs.core.windows.net/cms_audit_odag_process_start.txt") \
    .withColumn("FILE_NAME",input_file_name())

# Extract the file name from the full file path
df = df.withColumn("FILE_NAME", regexp_extract("FILE_NAME", r"([^/]+)$", 0))

In [0]:
df.createOrReplaceGlobalTempView("status_tracker_gtv")
display(df)

### Gold Layer

In [0]:
%sql
--DROP TABLE odag.gold.CMS_AUDIT_APP_STATUS_TRACKER;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS odag.gold.CMS_AUDIT_APP_STATUS_TRACKER
(
REQUEST_ID STRING,
PROTOCOL_YEAR STRING,
UNIVERSE_NUMBER STRING,
COMMENTS STRING,
LAST_UPDATED STRING  
);

In [0]:
%sql
INSERT INTO odag.gold.CMS_AUDIT_APP_STATUS_TRACKER
(REQUEST_ID, PROTOCOL_YEAR, UNIVERSE_NUMBER, COMMENTS, LAST_UPDATED)
SELECT 
REQUEST_ID,
PROTOCOL_YEAR,
UNIVERSE_NUMBER,
"Job is in progress" AS COMMENTS,
date_format(current_timestamp(), 'MM/dd/yyyy h:mm:ss a') AS LAST_UPDATED  -- Store formatted timestamp as string
FROM global_temp.status_tracker_gtv;

In [0]:
%sql
SELECT * FROM odag.gold.CMS_AUDIT_APP_STATUS_TRACKER;